# Load packages

In [1]:
pip install matplotlib numpy datasets transformers torch scikit-learn tqdm tabulate openai bitsandbytes accelerate\>=0.26.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.5/102.5 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 46.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 139.2 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 173.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 281.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

# Hugginface Login

In [2]:
API_KEY="please_replace_with_your_key"

from huggingface_hub import login

# Paste your Hugging Face token when prompted
login(API_KEY)

# Main code

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import datasets
import transformers
import re
import torch
import torch.nn.functional as F
import tqdm
import random
from sklearn.metrics import roc_curve, precision_recall_curve, auc
import argparse
import datetime
import os
import json
import functools
import custom_datasets
from multiprocessing.pool import ThreadPool
import time




# Generate a repeated version of the tab10 color map for 20 colors
COLORS = [plt.get_cmap('tab20')(i % 20) for i in range(20)]


# define regex to match all <extra_id_*> tokens, where * is an integer
pattern = re.compile(r"<extra_id_\d+>")


def load_base_model():
    print('MOVING BASE MODEL TO GPU...', end='', flush=True)
    start = time.time()
    try:
        mask_model.cpu()
    except NameError:
        pass
    if args.openai_model is None:
        base_model.to(DEVICE)
    print(f'DONE ({time.time() - start:.2f}s)')


def load_mask_model():
    print('MOVING MASK MODEL TO GPU...', end='', flush=True)
    start = time.time()

    if args.openai_model is None:
        base_model.cpu()
    if not args.random_fills:
        mask_model.to(DEVICE)
    print(f'DONE ({time.time() - start:.2f}s)')


def tokenize_and_mask(text, span_length, pct, ceil_pct=False):
    tokens = text.split(' ')
    mask_string = '<<<mask>>>'

    n_spans = pct * len(tokens) / (span_length + args.buffer_size * 2)
    if ceil_pct:
        n_spans = np.ceil(n_spans)
    n_spans = int(n_spans)

    n_masks = 0
    while n_masks < n_spans:
        start = np.random.randint(0, len(tokens) - span_length)
        end = start + span_length
        search_start = max(0, start - args.buffer_size)
        search_end = min(len(tokens), end + args.buffer_size)
        if mask_string not in tokens[search_start:search_end]:
            tokens[start:end] = [mask_string]
            n_masks += 1
    
    # replace each occurrence of mask_string with <extra_id_NUM>, where NUM increments
    num_filled = 0
    for idx, token in enumerate(tokens):
        if token == mask_string:
            tokens[idx] = f'<extra_id_{num_filled}>'
            num_filled += 1
    assert num_filled == n_masks, f"num_filled {num_filled} != n_masks {n_masks}"
    text = ' '.join(tokens)
    return text


def count_masks(texts):
    return [len([x for x in text.split() if x.startswith("<extra_id_")]) for text in texts]


# replace each masked span with a sample from T5 mask_model
def replace_masks(texts):
    n_expected = count_masks(texts)
    stop_id = mask_tokenizer.encode(f"<extra_id_{max(n_expected)}>")[0]
    tokens = mask_tokenizer(texts, return_tensors="pt", padding=True).to(DEVICE)
    outputs = mask_model.generate(**tokens, max_length=150, do_sample=True, top_p=args.mask_top_p, num_return_sequences=1, eos_token_id=stop_id)
    return mask_tokenizer.batch_decode(outputs, skip_special_tokens=False)


def extract_fills(texts):
    # remove <pad> from beginning of each text
    texts = [x.replace("<pad>", "").replace("</s>", "").strip() for x in texts]

    # return the text in between each matched mask token
    extracted_fills = [pattern.split(x)[1:-1] for x in texts]

    # remove whitespace around each fill
    extracted_fills = [[y.strip() for y in x] for x in extracted_fills]

    return extracted_fills


def apply_extracted_fills(masked_texts, extracted_fills):
    # split masked text into tokens, only splitting on spaces (not newlines)
    tokens = [x.split(' ') for x in masked_texts]

    n_expected = count_masks(masked_texts)

    # replace each mask token with the corresponding fill
    for idx, (text, fills, n) in enumerate(zip(tokens, extracted_fills, n_expected)):
        if len(fills) < n:
            tokens[idx] = []
        else:
            for fill_idx in range(n):
                text[text.index(f"<extra_id_{fill_idx}>")] = fills[fill_idx]

    # join tokens back into text
    texts = [" ".join(x) for x in tokens]
    return texts


def perturb_texts_(texts, span_length, pct, ceil_pct=False):
    if not args.random_fills:
        masked_texts = [tokenize_and_mask(x, span_length, pct, ceil_pct) for x in texts]
        raw_fills = replace_masks(masked_texts)
        extracted_fills = extract_fills(raw_fills)
        perturbed_texts = apply_extracted_fills(masked_texts, extracted_fills)

        # Handle the fact that sometimes the model doesn't generate the right number of fills and we have to try again
        attempts = 1
        while '' in perturbed_texts:
            idxs = [idx for idx, x in enumerate(perturbed_texts) if x == '']
            print(f'WARNING: {len(idxs)} texts have no fills. Trying again [attempt {attempts}].')
            masked_texts = [tokenize_and_mask(x, span_length, pct, ceil_pct) for idx, x in enumerate(texts) if idx in idxs]
            raw_fills = replace_masks(masked_texts)
            extracted_fills = extract_fills(raw_fills)
            new_perturbed_texts = apply_extracted_fills(masked_texts, extracted_fills)
            for idx, x in zip(idxs, new_perturbed_texts):
                perturbed_texts[idx] = x
            attempts += 1
    else:
        if args.random_fills_tokens:
            # tokenize base_tokenizer
            tokens = base_tokenizer(texts, return_tensors="pt", padding=True).to(DEVICE)
            valid_tokens = tokens.input_ids != base_tokenizer.pad_token_id
            replace_pct = args.pct_words_masked * (args.span_length / (args.span_length + 2 * args.buffer_size))

            # replace replace_pct of input_ids with random tokens
            random_mask = torch.rand(tokens.input_ids.shape, device=DEVICE) < replace_pct
            random_mask &= valid_tokens
            random_tokens = torch.randint(0, base_tokenizer.vocab_size, (random_mask.sum(),), device=DEVICE)
            # while any of the random tokens are special tokens, replace them with random non-special tokens
            while any(base_tokenizer.decode(x) in base_tokenizer.all_special_tokens for x in random_tokens):
                random_tokens = torch.randint(0, base_tokenizer.vocab_size, (random_mask.sum(),), device=DEVICE)
            tokens.input_ids[random_mask] = random_tokens
            perturbed_texts = base_tokenizer.batch_decode(tokens.input_ids, skip_special_tokens=True)
        else:
            masked_texts = [tokenize_and_mask(x, span_length, pct, ceil_pct) for x in texts]
            perturbed_texts = masked_texts
            # replace each <extra_id_*> with args.span_length random words from FILL_DICTIONARY
            for idx, text in enumerate(perturbed_texts):
                filled_text = text
                for fill_idx in range(count_masks([text])[0]):
                    fill = random.sample(FILL_DICTIONARY, span_length)
                    filled_text = filled_text.replace(f"<extra_id_{fill_idx}>", " ".join(fill))
                assert count_masks([filled_text])[0] == 0, "Failed to replace all masks"
                perturbed_texts[idx] = filled_text

    return perturbed_texts


def perturb_texts(texts, span_length, pct, ceil_pct=False):
    chunk_size = args.chunk_size
    if '11b' in mask_filling_model_name:
        chunk_size //= 2

    outputs = []
    for i in tqdm.tqdm(range(0, len(texts), chunk_size), desc="Applying perturbations"):
        outputs.extend(perturb_texts_(texts[i:i + chunk_size], span_length, pct, ceil_pct=ceil_pct))
    return outputs


def drop_last_word(text):
    return ' '.join(text.split(' ')[:-1])


def _openai_sample(p):
    if args.dataset != 'pubmed':  # keep Answer: prefix for pubmed
        p = drop_last_word(p)

    # sample from the openai model
    kwargs = { "engine": args.openai_model, "max_tokens": 200 }
    if args.do_top_p:
        kwargs['top_p'] = args.top_p
    
    r = openai.Completion.create(prompt=f"{p}", **kwargs)
    return p + r['choices'][0].text


# sample from base_model using ****only**** the first 30 tokens in each example as context
def sample_from_model(texts, min_words=55, prompt_tokens=30):
    # encode each text as a list of token ids
    if args.dataset == 'pubmed':
        texts = [t[:t.index(custom_datasets.SEPARATOR)] for t in texts]
        all_encoded = base_tokenizer(texts, return_tensors="pt", padding=True).to(DEVICE)
    else:
        all_encoded = base_tokenizer(texts, return_tensors="pt", padding=True).to(DEVICE)
        all_encoded = {key: value[:, :prompt_tokens] for key, value in all_encoded.items()}

    if args.openai_model:
        # decode the prefixes back into text
        prefixes = base_tokenizer.batch_decode(all_encoded['input_ids'], skip_special_tokens=True)
        pool = ThreadPool(args.batch_size)

        decoded = pool.map(_openai_sample, prefixes)
    else:
        decoded = ['' for _ in range(len(texts))]

        # sample from the model until we get a sample with at least min_words words for each example
        # this is an inefficient way to do this (since we regenerate for all inputs if just one is too short), but it works
        tries = 0
        while (m := min(len(x.split()) for x in decoded)) < min_words:
            if tries != 0:
                print()
                print(f"min words: {m}, needed {min_words}, regenerating (try {tries})")

            sampling_kwargs = {}
            if args.do_top_p:
                sampling_kwargs['top_p'] = args.top_p
            elif args.do_top_k:
                sampling_kwargs['top_k'] = args.top_k
            min_length = 50 if args.dataset in ['pubmed'] else 150
            outputs = base_model.generate(**all_encoded, min_length=min_length, max_length=200, do_sample=True, **sampling_kwargs, pad_token_id=base_tokenizer.eos_token_id, eos_token_id=base_tokenizer.eos_token_id)
            decoded = base_tokenizer.batch_decode(outputs, skip_special_tokens=True)
            tries += 1

    if args.openai_model:
        global API_TOKEN_COUNTER

        # count total number of tokens with GPT2_TOKENIZER
        total_tokens = sum(len(GPT2_TOKENIZER.encode(x)) for x in decoded)
        API_TOKEN_COUNTER += total_tokens

    return decoded


def get_likelihood(logits, labels):
    assert logits.shape[0] == 1
    assert labels.shape[0] == 1

    logits = logits.view(-1, logits.shape[-1])[:-1]
    labels = labels.view(-1)[1:]
    log_probs = torch.nn.functional.log_softmax(logits, dim=-1)
    log_likelihood = log_probs.gather(dim=-1, index=labels.unsqueeze(-1)).squeeze(-1)
    return log_likelihood.mean()



def get_ll(text):
    with torch.no_grad():
        tokenized = base_tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(DEVICE)
        labels = tokenized.input_ids
        outputs = base_model(**tokenized)
        logits = outputs.logits[:, :-1, :]
        labels = labels[:, 1:]  # Shift labels for autoregressive modeling
        
        # Compute log-likelihood manually
        log_probs = F.log_softmax(logits, dim=-1)
        ll_per_token = log_probs.gather(2, labels.unsqueeze(-1)).squeeze(-1)

        # Mask out padding tokens
        attention_mask = tokenized.attention_mask[:, 1:]
        valid_tokens = attention_mask.sum(dim=1)  # Count non-padding tokens
        ll_per_sample = (ll_per_token * attention_mask).sum(dim=1) / valid_tokens  # Normalize

        return ll_per_sample.mean().item()  # Average over batch



def get_lls(texts):
    if not args.openai_model:
        return [get_ll(text) for text in texts]
    else:
        global API_TOKEN_COUNTER

        # use GPT2_TOKENIZER to get total number of tokens
        total_tokens = sum(len(GPT2_TOKENIZER.encode(text)) for text in texts)
        API_TOKEN_COUNTER += total_tokens * 2  # multiply by two because OpenAI double-counts echo_prompt tokens

        pool = ThreadPool(args.batch_size)
        return pool.map(get_ll, texts)


# get the average rank of each observed token sorted by model likelihood
def get_rank(text, log=False):
    assert args.openai_model is None, "get_rank not implemented for OpenAI models"

    with torch.no_grad():
        tokenized = base_tokenizer(text, return_tensors="pt").to(DEVICE)
        logits = base_model(**tokenized).logits[:,:-1]
        labels = tokenized.input_ids[:,1:]

        # get rank of each label token in the model's likelihood ordering
        matches = (logits.argsort(-1, descending=True) == labels.unsqueeze(-1)).nonzero()

        assert matches.shape[1] == 3, f"Expected 3 dimensions in matches tensor, got {matches.shape}"

        ranks, timesteps = matches[:,-1], matches[:,-2]

        # make sure we got exactly one match for each timestep in the sequence
        assert (timesteps == torch.arange(len(timesteps)).to(timesteps.device)).all(), "Expected one match per timestep"

        ranks = ranks.float() + 1 # convert to 1-indexed rank
        if log:
            ranks = torch.log(ranks)

        return ranks.float().mean().item()


# get average entropy of each token in the text
def get_entropy(text):
    with torch.no_grad():
        tokenized = base_tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(DEVICE)
        logits = base_model(**tokenized).logits[:, :-1, :]

        probs = F.softmax(logits, dim=-1)
        log_probs = F.log_softmax(logits, dim=-1)
        entropy_per_token = -(probs * log_probs).sum(dim=-1)

        # Mask padding tokens
        attention_mask = tokenized.attention_mask[:, 1:]
        valid_tokens = attention_mask.sum(dim=1)  # Count non-padding tokens
        entropy_per_sample = (entropy_per_token * attention_mask).sum(dim=1) / valid_tokens  # Normalize

        return entropy_per_sample.mean().item()


def get_roc_metrics(real_preds, sample_preds, target_fpr=0.01):
    fpr, tpr, _ = roc_curve([0] * len(real_preds) + [1] * len(sample_preds), real_preds + sample_preds)
    roc_auc = auc(fpr, tpr)

    # Find the TPR at the given target FPR
    idx = np.where(fpr <= target_fpr)[0][-1]  # Last index where FPR is below threshold
    tp_at_low_fpr = tpr[idx]

    
    return fpr.tolist(), tpr.tolist(), float(roc_auc), float(tp_at_low_fpr)


def get_precision_recall_metrics(real_preds, sample_preds):
    precision, recall, _ = precision_recall_curve([0] * len(real_preds) + [1] * len(sample_preds), real_preds + sample_preds)
    pr_auc = auc(recall, precision)
    return precision.tolist(), recall.tolist(), float(pr_auc)


# save the ROC curve for each experiment, given a list of output dictionaries, one for each experiment, using colorblind-friendly colors
# save the ROC curve for each experiment, given a list of output dictionaries, one for each experiment, using colorblind-friendly colors
def save_roc_curves(experiments):
    plt.figure(figsize=(10, 7))
    plt.clf()

    # Define lists of line styles and markers to use cyclically
    line_styles = ['--']#['-', '--', '-.', ':']
    markers = ['o', 's', 'D', '^', 'v', 'P', '*', 'X', 'H', 'p', '>']  # A variety of marker styles

    for idx, (experiment, color) in enumerate(zip(experiments, COLORS)):
        metrics = experiment["metrics"]
        plt.plot(
            metrics["fpr"], 
            metrics["tpr"], 
            label=f"{experiment['name']} (AUC={metrics['roc_auc']:.3f})",
            color=color, 
            linestyle=line_styles[idx % len(line_styles)],  # Cycle through line styles
            marker=markers[idx % len(markers)],  # Cycle through markers
            markevery=0.1,  # Adds markers at regular intervals (every 10% of data points)
            linewidth=2,
            markersize=10
        )
        print(f"{experiment['name']} roc_auc: {metrics['roc_auc']:.3f}")

    plt.plot([0, 1], [0, 1], color='grey', lw=2, linestyle='--')

    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    
    plt.xlabel('False Positive Rate', fontsize=12)
    plt.ylabel('True Positive Rate', fontsize=12)
    plt.title(f'ROC Curves ({base_model_name} - {args.mask_filling_model_name})', fontsize=14)

    plt.grid(True)
    plt.legend(loc="center left", bbox_to_anchor=(1, 0.5), fontsize=10)
    plt.tight_layout()

    plt.savefig(f"{SAVE_FOLDER}/roc_curves.png", dpi=1200)


# save the histogram of log likelihoods in two side-by-side plots, one for real and real perturbed, and one for sampled and sampled perturbed
def save_ll_histograms(experiments):
    # first, clear plt
    plt.clf()

    for experiment in experiments:
        try:
            results = experiment["raw_results"]
            # plot histogram of sampled/perturbed sampled on left, original/perturbed original on right
            plt.figure(figsize=(20, 6))
            plt.subplot(1, 2, 1)
            plt.hist([r["sampled_ll"] for r in results], alpha=0.5, bins='auto', label='sampled')
            plt.hist([r["perturbed_sampled_ll"] for r in results], alpha=0.5, bins='auto', label='perturbed sampled')
            plt.xlabel("log likelihood")
            plt.ylabel('count')
            plt.legend(loc='upper right')
            plt.subplot(1, 2, 2)
            plt.hist([r["original_ll"] for r in results], alpha=0.5, bins='auto', label='original')
            plt.hist([r["perturbed_original_ll"] for r in results], alpha=0.5, bins='auto', label='perturbed original')
            plt.xlabel("log likelihood")
            plt.ylabel('count')
            plt.legend(loc='upper right')
            plt.savefig(f"{SAVE_FOLDER}/ll_histograms_{experiment['name']}.png")
        except:
            pass


# save the histograms of log likelihood ratios in two side-by-side plots, one for real and real perturbed, and one for sampled and sampled perturbed
def save_llr_histograms(experiments):
    # first, clear plt
    plt.clf()

    for experiment in experiments:
        try:
            results = experiment["raw_results"]
            # plot histogram of sampled/perturbed sampled on left, original/perturbed original on right
            plt.figure(figsize=(20, 6))
            plt.subplot(1, 2, 1)

            # compute the log likelihood ratio for each result
            for r in results:
                r["sampled_llr"] = r["sampled_ll"] - r["perturbed_sampled_ll"]
                r["original_llr"] = r["original_ll"] - r["perturbed_original_ll"]
            
            plt.hist([r["sampled_llr"] for r in results], alpha=0.5, bins='auto', label='sampled')
            plt.hist([r["original_llr"] for r in results], alpha=0.5, bins='auto', label='original')
            plt.xlabel("log likelihood ratio")
            plt.ylabel('count')
            plt.legend(loc='upper right')
            plt.savefig(f"{SAVE_FOLDER}/llr_histograms_{experiment['name']}.png")
        except:
            pass


def get_perturbation_results(span_length=10, n_perturbations=1, n_samples=500):
    load_mask_model()

    torch.manual_seed(0)
    np.random.seed(0)

    results = []
    original_text = data["original"]
    sampled_text = data["sampled"]

    perturb_fn = functools.partial(perturb_texts, span_length=span_length, pct=args.pct_words_masked)

    p_sampled_text = perturb_fn([x for x in sampled_text for _ in range(n_perturbations)])
    p_original_text = perturb_fn([x for x in original_text for _ in range(n_perturbations)])
    for _ in range(n_perturbation_rounds - 1):
        try:
            p_sampled_text, p_original_text = perturb_fn(p_sampled_text), perturb_fn(p_original_text)
        except AssertionError:
            break

    assert len(p_sampled_text) == len(sampled_text) * n_perturbations, f"Expected {len(sampled_text) * n_perturbations} perturbed samples, got {len(p_sampled_text)}"
    assert len(p_original_text) == len(original_text) * n_perturbations, f"Expected {len(original_text) * n_perturbations} perturbed samples, got {len(p_original_text)}"

    for idx in range(len(original_text)):
        results.append({
            "original": original_text[idx],
            "sampled": sampled_text[idx],
            "perturbed_sampled": p_sampled_text[idx * n_perturbations: (idx + 1) * n_perturbations],
            "perturbed_original": p_original_text[idx * n_perturbations: (idx + 1) * n_perturbations]
        })

    load_base_model()

    for res in tqdm.tqdm(results, desc="Computing log likelihoods"):
        p_sampled_ll = get_lls(res["perturbed_sampled"])
        p_original_ll = get_lls(res["perturbed_original"])
        res["original_ll"] = get_ll(res["original"])
        res["sampled_ll"] = get_ll(res["sampled"])
        res["all_perturbed_sampled_ll"] = p_sampled_ll
        res["all_perturbed_original_ll"] = p_original_ll
        res["perturbed_sampled_ll"] = np.mean(p_sampled_ll)
        res["perturbed_original_ll"] = np.mean(p_original_ll)
        res["perturbed_sampled_ll_std"] = np.std(p_sampled_ll) if len(p_sampled_ll) > 1 else 1
        res["perturbed_original_ll_std"] = np.std(p_original_ll) if len(p_original_ll) > 1 else 1

    return results


def run_perturbation_experiment(results, criterion, span_length=10, n_perturbations=1, n_samples=500):
    # compute diffs with perturbed
    predictions = {'real': [], 'samples': []}
    for res in results:
        if criterion == 'd':
            predictions['real'].append(res['original_ll'] - res['perturbed_original_ll'])
            predictions['samples'].append(res['sampled_ll'] - res['perturbed_sampled_ll'])
        elif criterion == 'z':
            if res['perturbed_original_ll_std'] == 0:
                res['perturbed_original_ll_std'] = 1
                print("WARNING: std of perturbed original is 0, setting to 1")
                print(f"Number of unique perturbed original texts: {len(set(res['perturbed_original']))}")
                print(f"Original text: {res['original']}")
            if res['perturbed_sampled_ll_std'] == 0:
                res['perturbed_sampled_ll_std'] = 1
                print("WARNING: std of perturbed sampled is 0, setting to 1")
                print(f"Number of unique perturbed sampled texts: {len(set(res['perturbed_sampled']))}")
                print(f"Sampled text: {res['sampled']}")
            predictions['real'].append((res['original_ll'] - res['perturbed_original_ll']) / res['perturbed_original_ll_std'])
            predictions['samples'].append((res['sampled_ll'] - res['perturbed_sampled_ll']) / res['perturbed_sampled_ll_std'])

    fpr, tpr, roc_auc, tp_at_low_fpr = get_roc_metrics(predictions['real'], predictions['samples'], target_fpr=0.01)
    p, r, pr_auc = get_precision_recall_metrics(predictions['real'], predictions['samples'])
    name = f'perturbation_{n_perturbations}_{criterion}'
    print(f"{name} ROC AUC: {roc_auc}, PR AUC: {pr_auc}")    
    return {
        'name': name,
        'predictions': predictions,
        'info': {
            'pct_words_masked': args.pct_words_masked,
            'span_length': span_length,
            'n_perturbations': n_perturbations,
            'n_samples': n_samples,
        },
        'raw_results': results,
        'metrics': {
            'roc_auc': roc_auc,
            'fpr': fpr,
            'tpr': tpr,
            'tp_at_low_fpr': tp_at_low_fpr  # New key added
        },
        'pr_metrics': {
            'pr_auc': pr_auc,
            'precision': p,
            'recall': r,
        },
        'loss': 1 - pr_auc,
    }





def run_baseline_threshold_experiment(criterion_fn, name, n_samples=500):
    torch.manual_seed(0)
    np.random.seed(0)

    results = []
    for batch in tqdm.tqdm(range(n_samples // batch_size), desc=f"Computing {name} criterion"):
        original_text = data["original"][batch * batch_size:(batch + 1) * batch_size]
        sampled_text = data["sampled"][batch * batch_size:(batch + 1) * batch_size]

        for idx in range(len(original_text)):
            results.append({
                "original": original_text[idx],
                "original_crit": criterion_fn(original_text[idx]),
                "sampled": sampled_text[idx],
                "sampled_crit": criterion_fn(sampled_text[idx]),
            })

    # compute prediction scores for real/sampled passages
    predictions = {
        'real': [x["original_crit"] for x in results],
        'samples': [x["sampled_crit"] for x in results],
    }

    fpr, tpr, roc_auc, tp_at_low_fpr = get_roc_metrics(predictions['real'], predictions['samples'], target_fpr=0.01)
    p, r, pr_auc = get_precision_recall_metrics(predictions['real'], predictions['samples'])
    print(f"{name}_threshold ROC AUC: {roc_auc}, PR AUC: {pr_auc}, TP@low FPR: {tp_at_low_fpr}")
    return {
        'name': f'{name}_threshold',
        'predictions': predictions,
        'info': {
            'n_samples': n_samples,
        },
        'raw_results': results,
        'metrics': {
            'roc_auc': roc_auc,
            'fpr': fpr,
            'tpr': tpr,
            'tp_at_low_fpr': tp_at_low_fpr  # New key added
        },
        'pr_metrics': {
            'pr_auc': pr_auc,
            'precision': p,
            'recall': r,
        },
        'loss': 1 - pr_auc,
    }











# strip newlines from each example; replace one or more newlines with a single space
def strip_newlines(text):
    return ' '.join(text.split())


# trim to shorter length
def trim_to_shorter_length(texta, textb):
    # truncate to shorter of o and s
    shorter_length = min(len(texta.split(' ')), len(textb.split(' ')))
    texta = ' '.join(texta.split(' ')[:shorter_length])
    textb = ' '.join(textb.split(' ')[:shorter_length])
    return texta, textb


def truncate_to_substring(text, substring, idx_occurrence):
    # truncate everything after the idx_occurrence occurrence of substring
    assert idx_occurrence > 0, 'idx_occurrence must be > 0'
    idx = -1
    for _ in range(idx_occurrence):
        idx = text.find(substring, idx + 1)
        if idx == -1:
            return text
    return text[:idx]


def generate_samples(raw_data, batch_size):
    torch.manual_seed(42)
    np.random.seed(42)
    data = {
        "original": [],
        "sampled": [],
    }

    for batch in range(len(raw_data) // batch_size):
        print('Generating samples for batch', batch, 'of', len(raw_data) // batch_size)
        original_text = raw_data[batch * batch_size:(batch + 1) * batch_size]
        sampled_text = sample_from_model(original_text, min_words=30 if args.dataset in ['pubmed'] else 55)

        for o, s in zip(original_text, sampled_text):
            if args.dataset == 'pubmed':
                s = truncate_to_substring(s, 'Question:', 2)
                o = o.replace(custom_datasets.SEPARATOR, ' ')

            o, s = trim_to_shorter_length(o, s)

            # add to the data
            data["original"].append(o)
            data["sampled"].append(s)
    
    if args.pre_perturb_pct > 0:
        print(f'APPLYING {args.pre_perturb_pct}, {args.pre_perturb_span_length} PRE-PERTURBATIONS')
        load_mask_model()
        data["sampled"] = perturb_texts(data["sampled"], args.pre_perturb_span_length, args.pre_perturb_pct, ceil_pct=True)
        load_base_model()

    return data


def generate_data(dataset, key):
    # load data
    if dataset in custom_datasets.DATASETS:
        data = custom_datasets.load(dataset, cache_dir)
    else:
        data = datasets.load_dataset(dataset, split='train', cache_dir=cache_dir)[key]

    # get unique examples, strip whitespace, and remove newlines
    # then take just the long examples, shuffle, take the first 5,000 to tokenize to save time
    # then take just the examples that are <= 512 tokens (for the mask model)
    # then generate n_samples samples

    # remove duplicates from the data
    data = list(dict.fromkeys(data))  # deterministic, as opposed to set()

    # strip whitespace around each example
    data = [x.strip() for x in data]

    # remove newlines from each example
    data = [strip_newlines(x) for x in data]

    # try to keep only examples with > 250 words
    if dataset in ['writing', 'squad', 'xsum']:
        long_data = [x for x in data if len(x.split()) > 250]
        if len(long_data) > 0:
            data = long_data

    random.seed(0)
    random.shuffle(data)

    data = data[:5_000]

    # keep only examples with <= 512 tokens according to mask_tokenizer
    # this step has the extra effect of removing examples with low-quality/garbage content
    tokenized_data = preproc_tokenizer(data)
    data = [x for x, y in zip(data, tokenized_data["input_ids"]) if len(y) <= 512]

    # print stats about remainining data
    print(f"Total number of samples: {len(data)}")
    print(f"Average number of words: {np.mean([len(x.split()) for x in data])}")

    return generate_samples(data[:n_samples], batch_size=batch_size)


def load_base_model_and_tokenizer(name):
    if args.openai_model is None:
        print(f'Loading BASE model {args.base_model_name}...')
        base_model_kwargs = {}
        if 'gpt-j' in name or 'neox' in name:
            base_model_kwargs.update(dict(torch_dtype=torch.float16))
        if 'gpt-j' in name:
            base_model_kwargs.update(dict(revision='float16'))
        base_model = transformers.AutoModelForCausalLM.from_pretrained(name, **base_model_kwargs, cache_dir=cache_dir)
    else:
        base_model = None

    optional_tok_kwargs = {}
    if "facebook/opt-" in name:
        print("Using non-fast tokenizer for OPT")
        optional_tok_kwargs['fast'] = False
    if args.dataset in ['pubmed']:
        optional_tok_kwargs['padding_side'] = 'left'
    base_tokenizer = transformers.AutoTokenizer.from_pretrained(name, **optional_tok_kwargs, cache_dir=cache_dir)
    base_tokenizer.pad_token_id = base_tokenizer.eos_token_id

    return base_model, base_tokenizer


def eval_supervised(data, model):
    print(f'Beginning supervised evaluation with {model}...')
    detector = transformers.AutoModelForSequenceClassification.from_pretrained(model, cache_dir=cache_dir).to(DEVICE)
    tokenizer = transformers.AutoTokenizer.from_pretrained(model, cache_dir=cache_dir)

    real, fake = data['original'], data['sampled']

    with torch.no_grad():
        # get predictions for real
        real_preds = []
        for batch in tqdm.tqdm(range(len(real) // batch_size), desc="Evaluating real"):
            batch_real = real[batch * batch_size:(batch + 1) * batch_size]
            batch_real = tokenizer(batch_real, padding=True, truncation=True, max_length=512, return_tensors="pt").to(DEVICE)
            real_preds.extend(detector(**batch_real).logits.softmax(-1)[:,0].tolist())
        
        # get predictions for fake
        fake_preds = []
        for batch in tqdm.tqdm(range(len(fake) // batch_size), desc="Evaluating fake"):
            batch_fake = fake[batch * batch_size:(batch + 1) * batch_size]
            batch_fake = tokenizer(batch_fake, padding=True, truncation=True, max_length=512, return_tensors="pt").to(DEVICE)
            fake_preds.extend(detector(**batch_fake).logits.softmax(-1)[:,0].tolist())

    predictions = {
        'real': real_preds,
        'samples': fake_preds,
    }

    fpr, tpr, roc_auc, tp_at_low_fpr = get_roc_metrics(real_preds, fake_preds, target_fpr=0.01)
    p, r, pr_auc = get_precision_recall_metrics(real_preds, fake_preds)
    print(f"{model} ROC AUC: {roc_auc}, PR AUC: {pr_auc}")

    # free GPU memory
    del detector
    torch.cuda.empty_cache()
    return {
        'name': model,
        'predictions': predictions,
        'info': {
            'n_samples': n_samples,
        },
        'metrics': {
            'roc_auc': roc_auc,
            'fpr': fpr,
            'tpr': tpr,
            'tp_at_low_fpr': tp_at_low_fpr  # New key added
        },
        'pr_metrics': {
            'pr_auc': pr_auc,
            'precision': p,
            'recall': r,
        },
        'loss': 1 - pr_auc,
    }





# Attribution xsum

In [13]:
import os
import json
import random
import torch
import numpy as np
import datetime
import transformers
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from collections import defaultdict
from datasets import load_dataset

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

class Args:
    dataset = "xsum"
    dataset_key = "document"
    n_samples = 500
    batch_size = 50
    base_model_name = "run"
    cache_dir = "./cache"
    output_name = ""
    int8 = False
    half = True
    openai_model = None
    openai_key = None

args = Args()

START_DATE = datetime.datetime.now().strftime('%Y-%m-%d')
START_TIME = datetime.datetime.now().strftime('%H-%M-%S-%f')

base_model_name = args.base_model_name.replace('/', '_')
SAVE_FOLDER = f"./results/{args.output_name}{base_model_name}/{START_DATE}-{START_TIME}"
os.makedirs(SAVE_FOLDER, exist_ok=True)

with open(os.path.join(SAVE_FOLDER, "args.json"), "w") as f:
    json.dump(args.__dict__, f, indent=4)

cache_dir = args.cache_dir
os.makedirs(cache_dir, exist_ok=True)

def strip_newlines(text):
    return ' '.join(text.split())

def generate_data(dataset, key):
    data = load_dataset(dataset, split='train', cache_dir=cache_dir)[key]
    data = list(dict.fromkeys([x.strip() for x in data]))
    data = [strip_newlines(x) for x in data if len(x.split()) > 250]
    random.seed(0)
    random.shuffle(data)
    return data[:args.n_samples]

raw_data = generate_data(args.dataset, args.dataset_key)

def load_model_and_tokenizer(model_name):
    dtype = torch.float16 if args.half else torch.float32
    model = transformers.AutoModelForCausalLM.from_pretrained(
        model_name,
        cache_dir=cache_dir,
        torch_dtype=dtype,
        device_map="auto"
    )
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir)
    tokenizer.pad_token = tokenizer.eos_token
    return model, tokenizer


def get_ll(model, tokenizer, text, max_tokens=None):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(DEVICE)
    if max_tokens is not None:
        inputs['input_ids'] = inputs['input_ids'][:, :max_tokens]
        inputs['attention_mask'] = inputs['attention_mask'][:, :max_tokens]

    labels = inputs.input_ids.clone()
    outputs = model(**inputs)
    logits = outputs.logits[:, :-1, :]
    labels = labels[:, 1:]
    attention_mask = inputs.attention_mask[:, 1:]
    log_probs = torch.nn.functional.log_softmax(logits, dim=-1)
    ll_per_token = log_probs.gather(2, labels.unsqueeze(-1)).squeeze(-1)
    ll = (ll_per_token * attention_mask).sum(dim=1) / attention_mask.sum(dim=1)
    return ll.item()

def sample_from_model(texts, model, tokenizer):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True).to(DEVICE)
    input_ids = inputs.input_ids[:, :30]
    attention_mask = torch.ones_like(input_ids)
    outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=300,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

model_names = [
    "meta-llama/Llama-2-7b-chat-hf",
    "meta-llama/Meta-Llama-3-8B",
    "tiiuae/falcon-7b",
    "Qwen/Qwen3-32B"
]

samples = []
labels = []

for model_name in model_names:
    print(f"Sampling from {model_name}")
    model, tokenizer = load_model_and_tokenizer(model_name)
    model.eval()

    for i in range(0, len(raw_data), args.batch_size):
        batch = raw_data[i:i + args.batch_size]
        gen_texts = sample_from_model(batch, model, tokenizer)
        samples.extend(gen_texts)
        labels.extend([model_name] * len(gen_texts))

    del model
    torch.cuda.empty_cache()

with open(os.path.join(SAVE_FOLDER, "attribution_data.json"), "w") as f:
    json.dump({"samples": samples, "labels": labels}, f)



n = 300  # fixed prefix length
print(f"Evaluating at fixed prefix length: {n}")
ll_dict_n = defaultdict(list)

for model_name in model_names:
    model, tokenizer = load_model_and_tokenizer(model_name)
    model.eval()
    for text in samples:
        ll = get_ll(model, tokenizer, text, max_tokens=n)
        ll_dict_n[model_name].append(ll)
    del model
    torch.cuda.empty_cache()

preds_n = []
for i in range(len(samples)):
    model_lls = [ll_dict_n[m][i] for m in model_names]
    pred = model_names[np.argmax(model_lls)]
    preds_n.append(pred)

error_stats = {}
for model in model_names:
    tp = sum((p == l == model) for p, l in zip(preds_n, labels))
    fn = sum((p != model and l == model) for p, l in zip(preds_n, labels))
    fp = sum((p == model and l != model) for p, l in zip(preds_n, labels))

    total_non_model = sum(1 for l in labels if l != model)
    total_model = sum(1 for l in labels if l == model)

    type_i_rate = fp / total_non_model if total_non_model else 0
    type_ii_rate = fn / total_model if total_model else 0

    error_stats[model] = {'type_i': type_i_rate, 'type_ii': type_ii_rate}

# Report results
print(f"\nError statistics at prefix length {n}:")
for model in model_names:
    print(f"{model}:")
    print(f"  Type I Error Rate: {error_stats[model]['type_i']:.4f}")
    print(f"  Type II Error Rate: {error_stats[model]['type_ii']:.4f}")

Sampling from meta-llama/Llama-2-7b-chat-hf


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Sampling from meta-llama/Meta-Llama-3-8B


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/17 [00:00<?, ?it/s]


Error statistics at prefix length 300:
meta-llama/Llama-2-7b-chat-hf:
  Type I Error Rate: 0.0233
  Type II Error Rate: 0.0000
meta-llama/Meta-Llama-3-8B:
  Type I Error Rate: 0.0000
  Type II Error Rate: 0.1420
tiiuae/falcon-7b:
  Type I Error Rate: 0.0193
  Type II Error Rate: 0.0000
Qwen/Qwen3-32B:
  Type I Error Rate: 0.0047
  Type II Error Rate: 0.0000


# Attribution squad

In [14]:
import os
import json
import random
import torch
import numpy as np
import datetime
import transformers
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from collections import defaultdict
from datasets import load_dataset

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

class Args:
    dataset = "squad"
    dataset_key = "context"
    n_samples = 500
    batch_size = 50
    base_model_name = "run"
    cache_dir = "./cache"
    output_name = ""
    int8 = False
    half = True
    openai_model = None
    openai_key = None

args = Args()

START_DATE = datetime.datetime.now().strftime('%Y-%m-%d')
START_TIME = datetime.datetime.now().strftime('%H-%M-%S-%f')

base_model_name = args.base_model_name.replace('/', '_')
SAVE_FOLDER = f"./results/{args.output_name}{base_model_name}/{START_DATE}-{START_TIME}"
os.makedirs(SAVE_FOLDER, exist_ok=True)

with open(os.path.join(SAVE_FOLDER, "args.json"), "w") as f:
    json.dump(args.__dict__, f, indent=4)

cache_dir = args.cache_dir
os.makedirs(cache_dir, exist_ok=True)

def strip_newlines(text):
    return ' '.join(text.split())

def generate_data(dataset, key):
    data = load_dataset(dataset, split='train', cache_dir=cache_dir)[key]
    data = list(dict.fromkeys([x.strip() for x in data]))
    data = [strip_newlines(x) for x in data if len(x.split()) > 250]
    random.seed(0)
    random.shuffle(data)
    return data[:args.n_samples]

raw_data = generate_data(args.dataset, args.dataset_key)

def load_model_and_tokenizer(model_name):
    dtype = torch.float16 if args.half else torch.float32
    model = transformers.AutoModelForCausalLM.from_pretrained(
        model_name,
        cache_dir=cache_dir,
        torch_dtype=dtype,
        device_map="auto"
    )
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir)
    tokenizer.pad_token = tokenizer.eos_token
    return model, tokenizer


def get_ll(model, tokenizer, text, max_tokens=None):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(DEVICE)
    if max_tokens is not None:
        inputs['input_ids'] = inputs['input_ids'][:, :max_tokens]
        inputs['attention_mask'] = inputs['attention_mask'][:, :max_tokens]

    labels = inputs.input_ids.clone()
    outputs = model(**inputs)
    logits = outputs.logits[:, :-1, :]
    labels = labels[:, 1:]
    attention_mask = inputs.attention_mask[:, 1:]
    log_probs = torch.nn.functional.log_softmax(logits, dim=-1)
    ll_per_token = log_probs.gather(2, labels.unsqueeze(-1)).squeeze(-1)
    ll = (ll_per_token * attention_mask).sum(dim=1) / attention_mask.sum(dim=1)
    return ll.item()

def sample_from_model(texts, model, tokenizer):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True).to(DEVICE)
    input_ids = inputs.input_ids[:, :30]
    attention_mask = torch.ones_like(input_ids)
    outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=300,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

model_names = [
    "meta-llama/Llama-2-7b-chat-hf",
    "meta-llama/Meta-Llama-3-8B",
    "tiiuae/falcon-7b",
    "Qwen/Qwen3-32B"
]

samples = []
labels = []

for model_name in model_names:
    print(f"Sampling from {model_name}")
    model, tokenizer = load_model_and_tokenizer(model_name)
    model.eval()

    for i in range(0, len(raw_data), args.batch_size):
        batch = raw_data[i:i + args.batch_size]
        gen_texts = sample_from_model(batch, model, tokenizer)
        samples.extend(gen_texts)
        labels.extend([model_name] * len(gen_texts))

    del model
    torch.cuda.empty_cache()

with open(os.path.join(SAVE_FOLDER, "attribution_data.json"), "w") as f:
    json.dump({"samples": samples, "labels": labels}, f)



n = 300  # fixed prefix length
print(f"Evaluating at fixed prefix length: {n}")
ll_dict_n = defaultdict(list)

for model_name in model_names:
    model, tokenizer = load_model_and_tokenizer(model_name)
    model.eval()
    for text in samples:
        ll = get_ll(model, tokenizer, text, max_tokens=n)
        ll_dict_n[model_name].append(ll)
    del model
    torch.cuda.empty_cache()

preds_n = []
for i in range(len(samples)):
    model_lls = [ll_dict_n[m][i] for m in model_names]
    pred = model_names[np.argmax(model_lls)]
    preds_n.append(pred)

error_stats = {}
for model in model_names:
    tp = sum((p == l == model) for p, l in zip(preds_n, labels))
    fn = sum((p != model and l == model) for p, l in zip(preds_n, labels))
    fp = sum((p == model and l != model) for p, l in zip(preds_n, labels))

    total_non_model = sum(1 for l in labels if l != model)
    total_model = sum(1 for l in labels if l == model)

    type_i_rate = fp / total_non_model if total_non_model else 0
    type_ii_rate = fn / total_model if total_model else 0

    error_stats[model] = {'type_i': type_i_rate, 'type_ii': type_ii_rate}

# Report results
print(f"\nError statistics at prefix length {n}:")
for model in model_names:
    print(f"{model}:")
    print(f"  Type I Error Rate: {error_stats[model]['type_i']:.4f}")
    print(f"  Type II Error Rate: {error_stats[model]['type_ii']:.4f}")

Sampling from meta-llama/Llama-2-7b-chat-hf


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Sampling from meta-llama/Meta-Llama-3-8B


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/17 [00:00<?, ?it/s]


Error statistics at prefix length 300:
meta-llama/Llama-2-7b-chat-hf:
  Type I Error Rate: 0.0186
  Type II Error Rate: 0.0000
meta-llama/Meta-Llama-3-8B:
  Type I Error Rate: 0.0000
  Type II Error Rate: 0.1006
tiiuae/falcon-7b:
  Type I Error Rate: 0.0037
  Type II Error Rate: 0.0363
Qwen/Qwen3-32B:
  Type I Error Rate: 0.0233
  Type II Error Rate: 0.0000
